In [1]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
import gzip
import pickle
import numpy as np

In [2]:
learning_rate = 0.001
epochs = 30
batch_size = 120

In [5]:
#Cargamos los datos
f = gzip.open('mnist.pkl.gz', 'rb')
dataset = pickle.load(f,encoding="latin1")

In [7]:
dat=np.array(dataset)
print(dat[1,1].shape)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset

(10000,)


<ipython-input-7-9d04016ce3ca>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dat=np.array(dataset)


In [8]:
x_train.shape

(50000, 784)

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [10]:
print(y_train[10000])

3


In [11]:
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [12]:
y_trainc[10000]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [14]:
model = Sequential()
model.add(Dense(512, activation='sigmoid', input_shape=(784,)))
model.add(Dense(600,activation='tanh'))
model.add(Dense(200,activation='selu'))
model.add(Dense(600,activation='relu'))
model.add(Dense(100,activation='elu'))
model.add(Dense(600,activation='sigmoid'))
model.add(Dense(500,activation='exponential'))
model.add(Dense(num_classes, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               401920    
                                                                 
 dense_2 (Dense)             (None, 600)               307800    
                                                                 
 dense_3 (Dense)             (None, 200)               120200    
                                                                 
 dense_4 (Dense)             (None, 600)               120600    
                                                                 
 dense_5 (Dense)             (None, 100)               60100     
                                                                 
 dense_6 (Dense)             (None, 600)               60600     
                                                                 
 dense_7 (Dense)             (None, 500)              

In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
history = model.fit(x_train, y_trainc,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_testc)
                    )

Epoch 1/30
417/417 [==============================] - 18s 39ms/step - loss: 0.1916 - accuracy: 0.5451 - val_loss: 0.0648 - val_accuracy: 0.8814
Epoch 2/30
417/417 [==============================] - 16s 38ms/step - loss: 0.0522 - accuracy: 0.9089 - val_loss: 0.0442 - val_accuracy: 0.9260
Epoch 3/30
417/417 [==============================] - 18s 42ms/step - loss: 0.0338 - accuracy: 0.9422 - val_loss: 0.0335 - val_accuracy: 0.9435
Epoch 4/30
417/417 [==============================] - 16s 38ms/step - loss: 0.0255 - accuracy: 0.9569 - val_loss: 0.0272 - val_accuracy: 0.9535
Epoch 5/30
417/417 [==============================] - 16s 38ms/step - loss: 0.0201 - accuracy: 0.9656 - val_loss: 0.0238 - val_accuracy: 0.9589
Epoch 6/30
417/417 [==============================] - 15s 36ms/step - loss: 0.0171 - accuracy: 0.9718 - val_loss: 0.0194 - val_accuracy: 0.9680
Epoch 7/30
417/417 [==============================] - 15s 36ms/step - loss: 0.0144 - accuracy: 0.9753 - val_loss: 0.0185 - val_accuracy:

In [17]:
score = model.evaluate(x_test, y_testc, verbose=1) #evaluar la eficiencia del modelo
print(score)
a=model.predict(x_test) #predicción de la red entrenada
print(a.shape)
print(a[1])
print("resultado correcto:")
print(y_testc[1])


313/313 [==============================] - 2s 6ms/step - loss: 0.0225 - accuracy: 0.9796
[0.022458765655755997, 0.9796000123023987]
313/313 [==============================] - 3s 9ms/step
(10000, 10)
[2.2929068e-08 1.6679097e-07 9.9998760e-01 3.1038976e-06 1.5056989e-07
 2.9612657e-08 2.6435477e-05 7.3300663e-09 1.4211818e-10 5.1978469e-12]
resultado correcto:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
#Para guardar el modelo en disco
model.save("red.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
#para cargar la red:
modelo_cargado = tf.keras.models.load_model('red.h5')